# 飞书知识库技术原型

## 飞书全局变量

企业版本，是可以用 `tenant_access_token` 的。

但是之前相同代码测试个人版本，报错，wiki space 权限。

In [13]:
%%time

app_id = "cli_a65aebd9f5bbd00b"
app_secret = "rZa6tNZVO1iaEyQ08DQYlf53VY4PahXw"
space_id="7408517502861852676"

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 µs


## 创建飞书客户端

In [2]:
%%time

import lark_oapi as lark

client = lark.Client.builder() \
    .app_id(app_id) \
    .app_secret(app_secret) \
    .build()

CPU times: user 1.06 s, sys: 207 ms, total: 1.27 s
Wall time: 1.27 s


In [3]:
%%time

import requests
import json

# 飞书的 API 端点
url = "https://open.feishu.cn/open-apis/auth/v3/tenant_access_token/internal"

# 请求头
headers = {
    "Content-Type": "application/json; charset=utf-8"
}

# 请求体
payload = {
    "app_id": app_id,
    "app_secret": app_secret
}

# 发起请求获取 tenant_access_token
response = requests.post(url, headers=headers, data=json.dumps(payload))

# 解析响应
if response.status_code == 200:
    data = response.json()
    if data.get("code") == 0:
        tenant_access_token = data.get("tenant_access_token")
        print(f"Tenant Access Token: {tenant_access_token}")
    else:
        print(f"Error: {data.get('msg')}")
else:
    print(f"Failed to get token, status code: {response.status_code}")


Tenant Access Token: t-g1049olRKMZZJYKPUP7CS3OIJFHCVBNYU2VTF4TC
CPU times: user 5.29 ms, sys: 291 µs, total: 5.58 ms
Wall time: 126 ms


In [4]:
%%time

from lark_oapi.api.wiki.v2 import *

request: ListSpaceNodeRequest = ListSpaceNodeRequest.builder() \
        .space_id(space_id) \
        .build()

CPU times: user 49 µs, sys: 10 µs, total: 59 µs
Wall time: 60.8 µs


In [5]:
%%time

req_opt = lark.RequestOption.builder()\
    .tenant_access_token(tenant_access_token)\
    .headers(headers)\
    .build()

response: ListSpaceNodeResponse = client.wiki.v2.space_node.list(request)

CPU times: user 5.78 ms, sys: 2.84 ms, total: 8.61 ms
Wall time: 587 ms


In [6]:
%%time

if not response.success():
    lark.logger.error(
        f"client.wiki.v2.space_node.list failed, code: {response.code}, msg: {response.msg}, log_id: {response.get_log_id()}")
else:
    lark.logger.info(lark.JSON.marshal(response.data, indent=4))

CPU times: user 131 µs, sys: 24 µs, total: 155 µs
Wall time: 158 µs


In [12]:
response.data.items[0].obj_token

'CtNndKaggogsx9xzoJjc9JFcnbb'

In [14]:
response.data.items[0].obj_edit_time

'1724929899'